# 한국어의 특징

## 한국어는 문법상 동사의 활용이 난해하고 조사의 존재 때문에 고립어인 영어보다 형태소 단위로 AI에게 학습시키기 난해하다.
## 하지만 영어보다는 문자를 발음할 때의 규칙성이 더 높기 때문에 STT/TTS를 제작할 때는 음절 단위로 학습시키는게 더 유리하다.


# 이를 위한 함수 chr(), ord()
## chr() : 문자 하나를 유니코드 숫자 값으로 변환시키는 함수
#### 예1) chr('A') == 65
#### 예2) chr('가') == 0xAC00
## ord() : 숫자를 해당 유니코드 숫자 값을 가진 문자 하나로 변환시키는 함수
#### 예1) ord(97) == 'a'


# 한국어 유니코드 완성형의 특징
## 한국어 유니코드는 완성형으로 숫자 값이 배정되어 있으나 숫자값 배정에 규칙이 존재해서 초성, 중성, 종성 분리가 쉽다
## 공식 : (초성 * 21 + 중성) * 28 + 종성 + 0xAC00
#### 초성 :
['ㄱ','ㄲ','ㄴ','ㄷ','ㄸ','ㄹ','ㅁ','ㅂ','ㅃ','ㅅ','ㅆ','ㅇ','ㅈ','ㅉ','ㅊ','ㅋ','ㅌ','ㅍ','ㅎ'] (19개)
#### 중성 :
['ㅏ','ㅐ','ㅑ','ㅒ','ㅓ','ㅔ','ㅕ','ㅖ','ㅗ','ㅘ','ㅙ','ㅚ','ㅛ','ㅜ','ㅝ','ㅞ','ㅟ','ㅠ','ㅡ','ㅢ','ㅣ'] (21개)
#### 종성 : 
['','ㄱ','ㄲ','ㄳ','ㄴ','ㄵ','ㄶ','ㄷ','ㄹ','ㄺ','ㄻ','ㄼ','ㄽ','ㄾ','ㄿ','ㅀ','ㅁ','ㅂ','ㅄ','ㅅ','ㅆ','ㅇ','ㅈ','ㅊ','ㅋ','ㅌ','ㅍ','ㅎ'] (28개)
#### 기타 특문을 위한 라벨링 :
['not','pad','unk',' ','.','!','?'] (7개, 가변적)


### 특문의 not은 해당 문자가 특문이 아님을 표시하는 부분입니다.
### 총 68 + (특문 갯수) 개


## 이를 위한 Multi-hot-encoding
### 기존 One-hot-encoding과 달리 1의 값이 여러개인 인코딩 방식
#### 예시로 '건'이라는 글자는
#### 앞의 0~18번째 벡터 중 0번째가 1, 나머지가 0
#### 중간의 19~39번째 백터 중 22번째가 1, 나머지가 0
#### 뒤의 40~67번째 벡터 중 43번째가 1, 나머지가 0
#### 위와 같이 표현이 가능하다
#### 또한 특문, 공백을 위해 68번째 뒤에 기타 코드를 배정하는게 좋다
### Multi-hot-encoding을 위한 torch의 손실함수는 torch.nn.MultiLabelSoftMarginLoss 이다.






참고 문헌 : https://koreascience.or.kr/article/CFKO201832073079068.pdf

In [1]:
import os
import sys
sys.path.append(os.path.join(os.path.dirname(""), ".."))

import custom
import numpy

In [2]:
# 변환 작업을 함수화

def text_to_label(text : str) :
    label = []
    chars = list(text)
    spe_ = ['<not>','<pad>','<unk>',' ','.','!','?']
    for c in chars :
        encoding = numpy.zeros(68+len(spe_))
        if c in spe_ :
            encoding[spe_.index(c) + 68] += 1
            label.append(encoding)
            continue
        value = ord(c) - 0xAC00
        if value < 0 or value >= 19*21*28 :
            encoding[spe_.index('<unk>') + 68] += 1
            label.append(encoding)
            continue
        jong_ = value % 28
        jung_ = (value // 28) % 21
        cho_  = (value // 28) // 21
        encoding[cho_] += 1
        encoding[jung_+19] += 1
        encoding[jong_+40] += 1
        encoding[spe_.index('<not>')+68] += 1 #특문이 아님을 표시
        label.append(encoding)
    label = numpy.array(label)
    return label

def label_to_text(label, special_threshold = 0.5) :
    text = ""
    spe_ = ['<not>','<pad>','<unk>',' ','.','!','?']
    
    for v in label :
        idx_ = numpy.argmax(v[68:68+len(spe_)], axis = -1).item()
        if idx_ != spe_.index('<not>') : # 특문일 경우 (<not>)
            text += spe_[idx_]
            continue
            
        cho_  = numpy.argmax(v[0:19], axis = -1).item()
        jung_ = numpy.argmax(v[19:40], axis = -1).item()
        jong_ = numpy.argmax(v[40:68], axis = -1).item()
        
        value = (cho_ * 21 + jung_) * 28 + jong_ + 0xAC00
        text += chr(value)   
    return text


In [3]:
cho = ['ㄱ','ㄲ','ㄴ','ㄷ','ㄸ','ㄹ','ㅁ','ㅂ','ㅃ','ㅅ','ㅆ','ㅇ','ㅈ','ㅉ','ㅊ','ㅋ','ㅌ','ㅍ','ㅎ']
jung = ['ㅏ','ㅐ','ㅑ','ㅒ','ㅓ','ㅔ','ㅕ','ㅖ','ㅗ','ㅘ','ㅙ','ㅚ','ㅛ','ㅜ','ㅝ','ㅞ','ㅟ','ㅠ','ㅡ','ㅢ','ㅣ']
jong = ['','ㄱ','ㄲ','ㄳ','ㄴ','ㄵ','ㄶ','ㄷ','ㄹ','ㄺ','ㄻ','ㄼ','ㄽ','ㄾ','ㄿ','ㅀ','ㅁ','ㅂ','ㅄ','ㅅ','ㅆ','ㅇ','ㅈ','ㅊ','ㅋ','ㅌ','ㅍ','ㅎ']
spe = ['<not>','<pad>','<unk>',' ','.','!','?']

In [4]:
char = ord('건')
print(char)

44148


In [5]:
value = (cho.index('ㄱ') * 21 + jung.index('ㅓ')) * 28 + jong.index('ㄴ') + 0xAC00
value_char = chr(value)
print(value_char)

건


In [6]:
ori_text = '건강 하네요.A'
label = text_to_label(ori_text)
print(label)

text = label_to_text(label)
print(text)

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
  0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
  0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [7]:
ori_text = '건강 하네요.A'
label = custom.text_to_label(ori_text)
print(label)

text = custom.label_to_text(label)
print(text)

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
  0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
  0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.